# 线性层在VGG中的使用

In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("..\examples"))

import utils; reload(utils)
from utils import *

from __future__ import division,print_function
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from utils import plots, get_batches, plot_confusion_matrix, get_data

from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

Using Theano backend.


## 处理各种路径

In [2]:
pwd = str(os.getcwd()).replace('\\', '/').split('/')
pwd.pop()
pwd.pop()
rootpath = "/".join(pwd)
path = rootpath + "/data/dogscats/"
path = rootpath + "/data/dogscats/sample/"

In [3]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size = 16

## 各种函数

In [4]:
# 利用bcolz, 可以储存和读取转化过的numpy.array
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [5]:
# get_data在get_batches的基础上, 加入了一些参数, 并且把其结果(keras.preprocessing.image.DirectoryIterator), 拼接成np.array
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

In [10]:
# One-Hot-Encoding
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

## 处理数据

In [12]:
trn_batches = get_batches(path + 'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path + 'valid', shuffle=False, batch_size=batch_size)
print(trn_batches.classes[5:10])

val_labels = onehot(val_batches.classes)
trn_labels = onehot(trn_batches.classes)
print(trn_labels[:5])

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
[0 0 0 1 1]
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


## 增加线性层来修改模型
在vgg16最后加入线性层, finetune为适用于dogs_cats的模型

### 利用原有模型得到预测结果  

利用原模型对图片进行预测, 得到的是vgg16的1000个class对应的概率

In [6]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

#### 方法一
利用model.predict_generator, 即dogscats中的做法(vgg.test)

In [11]:
trn_features = model.predict_generator(trn_batches, trn_batches.nb_sample)
val_features = model.predict_generator(val_batches, val_batches.nb_sample)

WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [12]:
trn_features[:5]

array([[  4.3092e-07,   2.9331e-06,   2.0046e-06, ...,   3.3366e-08,   1.7239e-05,   1.1062e-02],
       [  1.2095e-07,   8.4052e-06,   8.2558e-07, ...,   2.0857e-07,   1.4938e-04,   3.0650e-03],
       [  1.5470e-04,   5.2150e-05,   1.8671e-04, ...,   5.3959e-05,   3.2555e-03,   3.2866e-03],
       [  2.6524e-05,   1.0107e-05,   6.6178e-05, ...,   4.0122e-05,   1.4490e-03,   3.1767e-03],
       [  1.3526e-07,   4.2185e-07,   2.8940e-05, ...,   9.7323e-08,   6.0627e-05,   4.9876e-04]], dtype=float32)

#### 方法二
利用model.predict, 需要传入numpy.array, 所以需要对get_batches的数据进行预先处理  

In [7]:
trn_data = get_data(path+'train')
val_data = get_data(path+'valid')

save_array(model_path+'train_data.bc', trn_data)
save_array(model_path+'valid_data.bc', val_data)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [18]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')
val_data.shape

(8L, 3L, 224L, 224L)

In [10]:
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

### 利用线性层拟合
线性层以vgg原有的输出作为输入, dogs&cats作为输出

In [11]:
linear_model = Sequential([Dense(2, input_shape=(1000,), activation='softmax')])
linear_model.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
linear_model.fit(trn_features, trn_labels, validation_data=(val_features, val_labels), nb_epoch=3, batch_size=4)

Train on 16 samples, validate on 8 samples
Epoch 1/3
16/16 [==============================] - 0s - loss: 0.6781 - acc: 0.5625 - val_loss: 0.6602 - val_acc: 0.7500
Epoch 2/3
16/16 [==============================] - 0s - loss: 0.6404 - acc: 0.9375 - val_loss: 0.6431 - val_acc: 0.8750
Epoch 3/3
16/16 [==============================] - 0s - loss: 0.6177 - acc: 1.0000 - val_loss: 0.6295 - val_acc: 0.8750


In [13]:
linear_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 2)             2002        dense_input_2[0][0]              
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
____________________________________________________________________________________________________


### 得到结果

In [22]:
# 获得预测的class
preds = linear_model.predict_classes(val_features, batch_size=batch_size)
# 获得预测的概率
probs = linear_model.predict_proba(val_features, batch_size=batch_size)[:,0]

NameError: name 'val_features' is not defined

In [15]:
print(preds)
print(probs)

[0 0 0 0 1 1 1 0]
[ 0.5741  0.5665  0.5733  0.5319  0.4901  0.4443  0.4954  0.5418]


In [13]:
val_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [24]:
filenames = val_batches.filenames

# Number of images to view for each visualization task
n_view = 4
# Helper function to plot images by index in the validation set:


def plots_idx(idx, titles=None):
    plots([image.load_img(path + 'valid/' + filenames[i]) for i in idx], titles=titles)

# permutation(x): Randomly permute a sequence, or return a permuted range.

#1. A few images predicted as dogs
dogs = np.where(preds==1)[0]
idx = permutation(dogs)[:n_view]
plots_idx(idx, probs[idx])

#1. A few images predicted as cats
cats = np.where(preds==0)[0]
idx = permutation(cats)[:n_view]
plots_idx(idx, probs[idx])

NameError: name 'preds' is not defined

## Retrain最后一层(真正finetune)

In [13]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [14]:
## 去掉vgg原有的最后一层(线性层), 并将其他层trainable设置为False, 以免训练的时候修改了
model.pop()
for layer in model.layers:
    layer.trainable=False
    
## 加入新的线性层, 输出形态为2个类别
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
## 进行拟合
trn_data = get_data(path+'train')
val_data = get_data(path+'valid')
test_data = get_data(path+'test')
model.fit(trn_data, trn_labels, validation_data=(val_data, val_labels), nb_epoch=3, batch_size=4)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Found 8 images belonging to 1 classes.
Train on 16 samples, validate on 8 samples
Epoch 1/3
16/16 [==============================] - 11s - loss: 1.0036 - acc: 0.7500 - val_loss: 3.2736 - val_acc: 0.7500
Epoch 2/3
16/16 [==============================] - 12s - loss: 0.7798 - acc: 0.9375 - val_loss: 1.1504 - val_acc: 0.8750
Epoch 3/3
16/16 [==============================] - 14s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1504 - val_acc: 0.8750


In [16]:
preds = model.predict_classes(test_data, batch_size=batch_size)
prods = model.predict_proba(test_data, batch_size=batch_size)

8/8 [==============================] - 4s


In [20]:
print(preds)
print(prods[:, 0])

[0 0 0 0 1 1 0 0]
[  1.0000e+00   1.0000e+00   1.0000e+00   1.0000e+00   2.9316e-08   1.6012e-13   1.0000e+00
   1.0000e+00]


In [15]:
??permutation

In [23]:
np.where(preds==val_labels[:,1])

(array([0, 1, 2, 3, 4, 5, 7], dtype=int64),)

In [17]:
??vgg.finetune